In [1]:
import pandas as pd

In [3]:
df = pd.read_excel("thousand_lines_bert_humedit_right_thresh.xlsx", engine="openpyxl")

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,title,content,source,headline,masked,tokenized,span_index,span,predicted,new span,predicted_score
0,103,352419,352484,206931,A politician used the n-word in front of black...,A Florida state senator has stepped ...,AllTheNewsKaggle,A politician used the N-Word in front of black...,A politician used the N - Word in front of bla...,"['a_DET', 'politician_NOUN', 'use_VERB', 'the_...","(11,)","('colleague_NOUN',)",A politician used the N - Word in front of bla...,sheep,0.999951
1,784,180705,180768,212833,Alicia Keys is done playing nice. Your phone i...,"NEW YORK — On a cool Manhattan night, ...",AllTheNewsComponentsOne,Alicia Keys is done playing nice. your phone i...,Alicia Keys is done playing [MASK] . your phon...,"['Alicia_PROPN', 'Keys_PROPN', 'be_AUX', 'do_V...","(5,)","('nice_ADJ',)",Alicia Keys is done playing bingo . your phone...,bingo,0.999843
2,785,129532,129594,143206,Rio has turned Hope Solo into public enemy No. 1,The women’s Olympic soccer tournament is movin...,AllTheNewsComponentsOne,Rio has turned hope solo into public enemy no. 1,Rio has turned hope solo into public enemy [MA...,"['Rio_PROPN', 'have_AUX', 'turn_VERB', 'hope_N...","(8,)","('no_INTJ',)",Rio has turned hope solo into public enemy chi...,chihuahua,0.999743
3,782,143664,143726,163896,Is This Going To Make Me A Different Person? G...,"[When you're facing a major life change, it he...",AllTheNewsComponentsOne,Is this going to make me a different person? g...,Is this going to make me a different person ? ...,"['be_AUX', 'this_DET', 'go_VERB', 'to_PART', '...","(11,)","('transition_NOUN',)",Is this going to make me a different person ? ...,monkey,0.999713
4,524,345313,345378,195762,British Prime Minister Theresa May just told t...,In her first major speech outlining London’s ...,AllTheNewsKaggle,British Prime Minister Theresa may just told t...,British Prime Minister Theresa may just [MASK]...,"['british_ADJ', 'Prime_PROPN', 'Minister_PROPN...","(6,)","('tell_VERB',)",British Prime Minister Theresa may just half t...,half,0.999440


In [5]:
df[df['new span'] == '']

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,title,content,source,headline,masked,tokenized,span_index,span,predicted,new span,predicted_score


In [12]:
headlines = [headline for headline in df.dropna(subset=["new span"])["predicted"]]

In [13]:
len(headlines)

724

Проверим, остались ли слова, на тему которых шутить нельзя:

In [25]:
stop_words = ["christ","crucifix","church","pastor","priest",
              "god ","islam","muslim","mullah","rabbi ","jew","mosque","synagogue","hebrew",
              "sex","rape","rapist","kill","hurrican","jihad","terror"
             "murder","victim","shoot","die","death","genocide",
             "homicid","suicid","violence","shoot","tortur","slave",
             "pedophil","maniac","racist","racism","feminism","feminist",
             "black lives matter","hate","assassin","death","immigrant","immigration",
             "holocaust","surviv","attack","accident","protest","shot","gun","molest",
             "abus","harass","alive","bible","jesus","burial","buried", " bury", "funeral"]

In [26]:
[headline for headline in headlines if sum([stop_word in headline for stop_word in stop_words])>0]

['Didi Gregorius keeps getting better — and just weaker buried',
 'People are going to prison thanks to jesus software — but how it works is secret']

Оставим только то, насчёт чего шутить можно:

In [28]:
headlines = [headline for headline in headlines if sum([stop_word in headline for stop_word in stop_words])==0]

In [29]:
with open("bert_humicroedit_headlines_right_thresh.tsv", "w", encoding="utf-8") as outp:
    outp.write("INPUT:headline\n")
    for headline in headlines[:-1]:
        outp.write(headline.strip('"')+'\n')
    outp.write(headlines[-1])

Возьмём также сабсет из реальных новостей:

In [3]:
df_real = pd.read_excel("unsupervised_news_truecased.xlsx", engine="openpyxl").dropna(subset=["title_true_case"])

In [4]:
df_subset = df_real.sample(n=20,random_state=42)

In [5]:
df_subset.to_csv("real_subset.tsv", sep='\t')

И золотой стандарт (там, где разметка совпала):

In [4]:
df_gold = pd.read_excel("FullAgreement.xlsx", engine="openpyxl", index_col="Unnamed: 0").dropna(subset=["predicted"])

In [5]:
df_gold.head()

,id,title_orig_case,source,headline,masked,predicted,new span,predicted_score,method,Humorousness,Grammaticality,Humorousness_b,Grammaticality_b
7,129372,US ship sent as warning to North Korea was sai...,AllTheNewsKaggle,Us ship sent as warning to North Korea was sai...,Us ship sent as warning to North Korea was sai...,Us ship sent as warning to North Korea was sai...,departed,0.836966,bert_humedit,0,3,0,3
11,125023,There’s something fishy about this Tribeca tow...,AllTheNewsComponentsOne,There ’ s something Fishy about this Tribeca T...,There ’ s something [MASK] about this Tribeca ...,There ’ s something magical about this Tribeca...,magical,0.000130,bert_humedit,0,3,0,3
19,119228,Will the Trump Nomination Change Our Polarized...,AllTheNewsKaggle,Will the Trump nomination change our polarized...,Will the Trump nomination change our polarized...,Will the Trump nomination change our polarized...,sleeping,0.000106,bert_humedit,1,3,1,3
36,171638,Embattled Pennsylvania Attorney General Resign...,AllTheNewsKaggle,Embattled Pennsylvania Attorney General resign...,Embattled Pennsylvania Attorney General [MASK]...,Embattled Pennsylvania Attorney General sacked...,sacked,0.000280,bert_humedit,0,3,0,3
47,100507,Spy Agencies Around The World Are Digging Int...,AllTheNewsComponentsOne,Spy agencies around the world are digging into...,Spy agencies around the world are digging into...,Spy agencies around the world are digging into...,satan,0.000285,bert_humedit,3,3,3,3


In [7]:
with open("gold_standard.tsv", "w", encoding="utf-8") as outp:
    outp.write("INPUT:headline\tGOLDEN:hum\tGOLDEN:gram")
    for index, row in df_gold.iterrows():
        outp.write('\n')
        item = row['predicted']
        hum = row['Humorousness']
        gram = row['Grammaticality']
        outp.write(f"{item}\t{hum}\t{gram}")